In [4]:
!pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
!pip install transformers
!pip install gradio
!pip install timm
!pip install timm
!pip install inflect
!pip install phonemizer

In [7]:
!pip install py-espeak-ng
!pip install espeakng
!pip install --upgrade transformers accelerate
# for my mac, $brew install espeakng worked instead.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 3.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [1]:
from transformers.utils import logging

logging.set_verbosity_error()

In [2]:
from transformers import VitsModel, AutoTokenizer
import torch

model = VitsModel.from_pretrained("kakao-enterprise/vits-ljs")
tokenizer = AutoTokenizer.from_pretrained("kakao-enterprise/vits-ljs")

2024-11-23 10:15:23.948505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [94]:
#text = "Based on your criteria of a moderate climate, around thirty four percent sunny days, a salary of one thousand dollars, and a beach environment, here are a few countries that might fit your preferences Portugal: Coastal regions have a Mediterranean climate with mild winters and warm summers. You can find affordable living options, especially in smaller towns.Also known for beautiful beaches and coastal scenery. Mexico (specifically the Yucatán Peninsula)have warm tropical climate, but coastal areas have breezy conditions. Has many sunny days, but thehumidity can be high. Affordable, with many expats living comfortably on one thousand dollars a month Also has stunning beaches like those in Playa del Carmen or Tulum."
text = "Based on your criteria of a moderate climate, around thirty four percent sunny days, a salary of one thousand dollars, and a beach environment, here are a few countries that might fit your preferences. Portugal, Coastal regions like the Algarve, Mexico specifically the Yucatán Peninsula, and Vietnam Da Nang or Nha Trang region."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    output = model(**inputs).waveform

In [95]:
from IPython.display import Audio as IpythonAudio
from IPython.display import FileLink
import numpy as np

#play output in the notebook
IpythonAudio(output, rate=model.config.sampling_rate)

/Users/ai_mechanic/anaconda3/lib/python3.10/site-packages/gradio/processing_utils.py:738: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [64]:
import os
import gradio as gr
#demo = gr.Blocks()

def text_to_speech(text):
    if text is None:
        gr.Warning("No text found, please retry.")
        return ""
    text = text
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        output = model(**inputs).waveform
    
    # Convert waveform to NumPy array
    waveform_np = output.numpy()

    # Get the sampling rate
    sampling_rate = model.config.sampling_rate

    # Return the tuple (sample_rate, audio_data)
    return sampling_rate, waveform_np.squeeze() 

In [65]:
text_to_speech("Hello from the other side")

(22050,
 array([ 1.1466037e-04,  1.4109076e-04, -2.7695889e-04, ...,
         3.0929694e-04,  2.3917113e-04,  2.3604453e-05], dtype=float32))

In [66]:
demo = gr.Interface(
    fn=text_to_speech,
    inputs=gr.Textbox(label="Input text"),
    outputs=gr.Audio(label="Converted Audio"), 
    allow_flagging="never"
)

/Users/ai_mechanic/anaconda3/lib/python3.10/site-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [97]:
demo.launch(share=False) 
            #server_port=int(os.environ['PORT1']))

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [96]:
demo.close()

Closing server running on port: 7863


Based on your criteria of a moderate climate, around thirty four percent sunny days, a salary of one thousand dollars, and a beach environment, here are a few countries that might fit your preferences. Portugal, Coastal regions like the Algarve. Mexico specifically the Yucatán Peninsula, and Vietnam Da Nang or Nha Trang region.